In [56]:
import json

# Load the JSON file
with open('/content/drive/MyDrive/SSMT/test_word_boundaries.json', 'r') as file:
    data = json.load(file)

# Initialize the lists for filenames, words, and timestamps
filenames_words_bound = []
words = []
time_stamps = []

# Extract words and time ranges per file
for filename, entries in data.items():
    filenames_words_bound.append(filename)  # Add the filename to the list
    file_words = []
    file_time_stamps = []

    for entry in entries:
        file_words.append(entry['word'])  # Add the word to the list for the current file
        file_time_stamps.append((entry['start_time'], entry['end_time']))  # Add the start and end times for the current file

    words.append(file_words)  # Append the list of words for the current file
    time_stamps.append(file_time_stamps)  # Append the list of time stamps for the current file

# Print the lists
print("Filenames:", filenames_words_bound)
print("Words per file:", words)
print("Start and End Times per file:", time_stamps)


Filenames: ['5111153817746112904.wav', '5196260640941390313.wav', '14077379425479054842.wav', '17541705185473334415.wav', '13031516147694203229.wav', '1146643745158029931.wav', '1116140877447157610.wav', '5910253289107526048.wav', '2024594662850694945.wav', '3111983050474671345.wav', '6243143230365031162.wav', '14738234113419638776.wav', '18359063029191000694.wav', '559615000143317446.wav', '18080893208509673354.wav', '10899527843662463911.wav', '1616366247253366320.wav', '7587276269543309929.wav', '10937407360976471523.wav', '17406400587485166719.wav', '13732033283623109766.wav', '16757205419144758552.wav', '1731561740051730459.wav', '11426556553252626005.wav', '11617272550749387741.wav', '1273817225649957097.wav', '6745531990699497785.wav', '14648010410081654131.wav', '5970882371928079940.wav', '10471564175308895403.wav', '7366992441583559574.wav', '6656552880161745727.wav', '3926276660303924456.wav', '14396239747345303015.wav', '7546471028199266685.wav', '8491962032427728850.wav', '

In [57]:
print(len(filenames_words_bound))

647


In [58]:
import scipy.io

# Load the .mat file
mat_data = scipy.io.loadmat('/content/drive/MyDrive/SSMT/test.mat')

# Print the keys of the dictionary to understand its structure
print(mat_data.keys())

# Assuming the features are stored under the key 'features',
# you may need to adjust the key based on the actual structure of the .mat file.
features = mat_data['features']  # Replace 'features' with the actual key name

# Check the shape of the features matrix
print("Features shape:", features.shape)

# Now you can use the features array as needed


dict_keys(['__header__', '__version__', '__globals__', 'filename', 'features'])
Features shape: (269434, 768)


In [59]:
print(mat_data['filename'].shape)

(1, 269434)


In [60]:
features = mat_data['features']
filenames = mat_data['filename'].flatten()
# print(filenames)

In [61]:
unique_elements = np.unique(filenames)
num_unique = len(unique_elements)

print("Number of unique elements:", num_unique)

Number of unique elements: 556


In [62]:
from tqdm import tqdm

features_grouped = []
filenames2 = []

for i, fname in tqdm(enumerate(filenames), total=269434, desc="Grouping features"):
    if fname not in filenames2:
        filenames2.append(fname)
        features_grouped.append([features[i]])
    else:
        idx = filenames2.index(fname)
        features_grouped[idx].append(features[i])


Grouping features: 100%|██████████| 269434/269434 [01:48<00:00, 2477.25it/s]


In [63]:
print(len(filenames2))
print(len(features_grouped))

556
556


In [64]:
features_grouped1 = [np.array(group) for group in features_grouped]

In [65]:
features_grouped1[0].T.shape

(768, 631)

In [66]:
len(filenames2)

556

In [67]:
len(filenames_words_bound)

647

In [68]:
len(words)


647

In [69]:
len(time_stamps)

647

In [70]:
import numpy as np
from tqdm import tqdm

# Assuming these are already defined:
# filenames: list of filenames (order of feature arrays)
# features: list of 2D numpy arrays, one per file
# filenames_words: list of filenames (matching with words_per_file and timestamps_per_file)
# words_per_file: list of list of words
# timestamps_per_file: list of list of (start_time, end_time) tuples

filenames = filenames2
features = features_grouped1
filenames_words_bound_cleaned = [f.replace('.wav', '') for f in filenames_words_bound]

def extract_syllable_features(features, start_time, end_time, stride=0.02):
    """Extracts and averages feature frames corresponding to a syllable."""
    start_frame = max(0, int(start_time / stride))
    end_frame = max(0, int(end_time / stride))
    return np.mean(features[start_frame:end_frame + 1, :], axis=0)

# Output lists
final_filenames = []
final_words = []
final_mean_features = []

# Iterate through the filenames in the feature list
for i, fname in tqdm(enumerate(filenames), total=len(filenames)):
    if fname in filenames_words_bound_cleaned:
        idx = filenames_words_bound_cleaned.index(fname)
        word_list = words[idx]
        time_list = time_stamps[idx]
        feature_array = features[i]

        mean_feats = []
        for (start, end) in time_list:
            mean_feat = extract_syllable_features(feature_array, start, end)
            mean_feats.append(mean_feat)

        final_filenames.append(fname)
        final_words.append(word_list)
        final_mean_features.append(mean_feats)

print(f"Final filenames: {final_filenames[:2]}")
print(f"Final words: {final_words[:2]}")
print(f"Final mean features: {len(final_mean_features[0])}" )

100%|██████████| 556/556 [00:00<00:00, 678.64it/s]

Final filenames: [array(['12741024238657315067'], dtype='<U20'), array(['9055240296608129734'], dtype='<U19')]
Final words: [[' Unfortunately,', ' stunning', ' traffic', ' flow', ' is', ' difficult', ' because', ' driver', ' behavior', ' cannot', ' be', ' predicted', ' with', ' 100', '%', ' certainty.'], [' Unfortunately,', ' studying', ' traffic', ' flow', ' is', ' difficult', ' because', ' driver', ' behavior', ' cannot', ' be', ' predicted', ' with', ' 100', '%', ' certainty.']]
Final mean features: 16


In [71]:
print(f"Final filenames: {final_filenames[0]}")
print(f"Final words: {final_words[0]}")
print(f"Final mean features: {len(final_mean_features[0])}" )

Final filenames: ['12741024238657315067']
Final words: [' Unfortunately,', ' stunning', ' traffic', ' flow', ' is', ' difficult', ' because', ' driver', ' behavior', ' cannot', ' be', ' predicted', ' with', ' 100', '%', ' certainty.']
Final mean features: 16


In [72]:
print(final_mean_features[0][0].shape)

(768,)


In [73]:
print(filenames[0])

['12741024238657315067']


In [74]:
print(filenames_words_bound)

['5111153817746112904.wav', '5196260640941390313.wav', '14077379425479054842.wav', '17541705185473334415.wav', '13031516147694203229.wav', '1146643745158029931.wav', '1116140877447157610.wav', '5910253289107526048.wav', '2024594662850694945.wav', '3111983050474671345.wav', '6243143230365031162.wav', '14738234113419638776.wav', '18359063029191000694.wav', '559615000143317446.wav', '18080893208509673354.wav', '10899527843662463911.wav', '1616366247253366320.wav', '7587276269543309929.wav', '10937407360976471523.wav', '17406400587485166719.wav', '13732033283623109766.wav', '16757205419144758552.wav', '1731561740051730459.wav', '11426556553252626005.wav', '11617272550749387741.wav', '1273817225649957097.wav', '6745531990699497785.wav', '14648010410081654131.wav', '5970882371928079940.wav', '10471564175308895403.wav', '7366992441583559574.wav', '6656552880161745727.wav', '3926276660303924456.wav', '14396239747345303015.wav', '7546471028199266685.wav', '8491962032427728850.wav', '40606705804

In [75]:
print(len(filenames_words_bound))

647


In [76]:
import scipy.io

# Save to .mat file with custom keys
scipy.io.savemat('/content/drive/MyDrive/SSMT/test_word_level_features.mat', {
    'word_features': final_mean_features,
    'words': final_words,
    'filenames': final_filenames
})


In [77]:
import scipy.io

# Load the .mat file
mat_data = scipy.io.loadmat('/content/drive/MyDrive/SSMT/test_word_level_features.mat')

# Print the keys of the dictionary to understand its structure
print(mat_data.keys())

# Assuming the features are stored under the key 'features',
# you may need to adjust the key based on the actual structure of the .mat file.
features = mat_data['word_features'][0][0]  # Replace 'features' with the actual key name

# Check the shape of the features matrix
print("Features shape:", features.shape)

# Now you can use the features array as needed


dict_keys(['__header__', '__version__', '__globals__', 'word_features', 'words', 'filenames'])
Features shape: (16, 768)
